In [1]:
import torch
import numpy as np
print(f'PyTorch version: {torch.__version__}')
print(f'Is MPS available? {torch.backends.mps.is_available()}')
print(f'Does MPS exists? {torch.backends.mps.is_built()}')

PyTorch version: 2.6.0
Is MPS available? True
Does MPS exists? True


# Introducción

*Las redes neuronales son modelos muy poderosos que nos permiten resolver una gran cantidad de problemas gracias a su capacidad para aprender/extraer patrones de los datos. Por ejemplo, estos modelos se pueden utilizar para detectar tumores en imágenes médicas o para transformar texto en audio.*

*Cómo podemos entrenar estos modelos? Con ejemplos. Supongamos que queremos entrenar una red neuronal que distingue entre perros y gatos. Lo primero que necesitamos es juntar muchas imágenes que contengan perros y gatos, y a cada una le tendríamos que agregar una etiqueta que indique si el animal de la imagen es un perro o es un gato. A partir de esta combinación de imágenes y etiquetas, el modelo podrá extraer patrones (i.e., aprender las características más comunes en los datos) que le permitirán discriminar entre las imágenes. Una vez entrenado, comparamos sus predicciones con las etiquetas reales, para luego ajustar sus parámetros con el objetivo de mejorar sus predicciones.*

*El problema es que las computadoras, a diferencia de los humanos, solo entienden de números. Entonces, para poder entrenar estos modelos, primero necesitamos transformar nuestros datos (i.e., imágenes, texto, audio, etc) en una estructura númerica que pueda ser interpretada por la computadora.*

*PyTorch nos proporciona una estructura de datos, los **tensores**, que nos permiten representar nuestros datos -ya sean imágenes, texto o audio- en números. A diferencia de su significado en matemática, los tensores en PyTorch se refieren a **arrays multidimensionales**, similares a los arrays de NumPy, pero con capacidades adicionales que los hacen especialmente útiles para entrenar modelos de Deep Learning*.

<img src="attachments/tensors-in-pytorch.png" width="1400" height="600" style="display: block; margin: 0 auto;" />

## Crear tensores

### Escalares

In [2]:
# Create a scalar
scalar = torch.tensor(7.0)
scalar

tensor(7.)

*Los tensores son arrays n-dimensionales. La dimensionalidad del tensor es equivalente a la cantidad de índices necesarios para obtener un escalar. Dicho esto, un escalar es un tensor de dimensión 0 (porque no tenemos que indexar el tensor para obtenerlo).*

In [6]:
scalar.ndim

0

*Para extraer el número del tensor podemos utilizar el método `item()`.*

In [7]:
# Get a tensor back as an integer or float
print(f'Escalar: {scalar.item()}. Type: {type(scalar.item())}')

Escalar: 7.0. Type: <class 'float'>


### Vectores

*Los vectores son tensores **uni-dimensionales**. Utilizando el atributo `.ndim` podemos ver que el tensor que creamos tiene una única dimensión (porque para obtener un escalar necesitamos un único índice).*

In [8]:
# Create a vector
vector = torch.tensor([7, 7])
print(f'Vector = {vector}')
print(f'Dimensions: {vector.ndim}')

Vector = tensor([7, 7])
Dimensions: 1


*Usando el atributo `.shape` podemos ver el tamaño de las dimensiones de nuestro tensor. En este caso, nuestro tensor solo tiene una dimensión con dos elementos.*

In [9]:
print(f'Shape: {vector.shape}')

Shape: torch.Size([2])


### Matrices

*Las matrices son tensores **bi-dimensionales**, en donde la primera dimensión representa las filas y la segunda dimensión representa las columnas. El tamaño de cada dimensión hace referencia a la cantidad de filas y a la cantidad de columnas de la matriz.*

In [13]:
MATRIX = torch.tensor([[7, 8], [8, 9]])
print(f'Matrix =\n {MATRIX}')
print(f'Dimensions: {MATRIX.ndim}')
print(f'Shape: {MATRIX.shape}')

Matrix =
 tensor([[7, 8],
        [8, 9]])
Dimensions: 2
Shape: torch.Size([2, 2])


### Tensores

*Si bien los vectores y las matrices son, al menos en PyTorch, tensores de 1D y 2D, respectivamente, llamamos tensor a los arrays **n-dimensionales** (con $n > 2$). Por ejemplo, debajo tenemos un tensor de tres dimensiones.*

In [14]:
# Create a tensor
TENSOR = torch.tensor([[[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]]])
TENSOR

tensor([[[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]])

In [15]:
print(f'Dimensions: {TENSOR.ndim}')
print(f'Shape: {TENSOR.shape}')

Dimensions: 3
Shape: torch.Size([1, 3, 3])


*Para crear un tensor, utilizando otro(s) tensor(es) como input(s), es necesario utilizar funciones como `torch.stack()` o `torch.vstack()`. La clase `torch.tensor()` solo acepta objetos nativos de Python. En este caso utilizamos la función `torch.stack()` que nos permite concatenar tensores sobre una nueva dimensión.*

In [16]:
A = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
B = torch.tensor([[10, 11, 12], [13, 14, 15], [16, 17, 18]])

TENSOR = torch.stack([A, B])

print(f'Dimension of A = {A.ndim}, Shape of A = {A.shape}')
print(f'Dimension of B = {B.ndim}, Shape of B = {B.shape}')
print(f'Dimension of TENSOR = {TENSOR.ndim}, Shape of TENSOR = {TENSOR.shape}')

Dimension of A = 2, Shape of A = torch.Size([3, 3])
Dimension of B = 2, Shape of B = torch.Size([3, 3])
Dimension of TENSOR = 3, Shape of TENSOR = torch.Size([2, 3, 3])


In [17]:
TENSOR

tensor([[[ 1,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9]],

        [[10, 11, 12],
         [13, 14, 15],
         [16, 17, 18]]])

## Tensores en memoria

*A diferencia de las estructuras de datos nativas de Python, como las listas, los tensores se guardan en memoria como un **único bloque contiguo**. Qué quiere decir esto? Que a diferencia de las listas, donde cada elemento es un objeto de Python que tiene su propio espacio en memoria, los tensores se encuentran guardados en un único espacio en memoria. Es decir, no importa cuantas dimensiones tenga nuestro tensor, en memoria se guarda como un vector uni-dimensional donde cada elemento se encuentra uno al lado del otro. Esto hace que sea mucho más rápido acceder secuencialmente a los elementos en un tensor y que sea mucho más sencillo vectorizar operaciones con ellos.*

*Esto está relacionado con la arquitectura de PyTorch. Cuando creamos una instancia de `torch.Tensor` también creamos una instancia de `torch.Storage`. Esta instancia de `torch.Storage` no es más que un vector uni-dimensional que contiene los datos en memoria. La instancia de `torch.Tensor`, que es con la que nosotros, los usuarios, interactuamos, es una vista de la instancia `torch.Storage`, es decir, una vista de los datos en memoria. Esto hace que PyTorch sea muy eficiente en el manejo de los datos, porque nos permite crear varios tensores sin tener que duplicar nuestros datos en memoria (i.e., solo necesitamos crear una nueva referencia al objeto en memoria).*

*La siguiente imagen es muy representación de como se guarda en memoria una lista (u otros objetos nativos de Python) y como se guardan los tensores:*
<figure>
    <img src="attachments/tensors-in-memory.png" width="1400" height="600" align="center" style="display: block; margin: 0 auto;"/>
    <figcaption style="text-align: center;">Stevens, Eli. (2020). Python object (boxed) numeric values versus tensor (unboxed array) numeric values. In Stevens, Eli, <i>Deep Learning with PyTorch</i> (p. 44).</figcaption>
</figure>

### Metadata

*Como ya dijimos, los tensores se guardan en memoria como vectores uni-dimensionales, aunque los hayamos creado con más de una dimensión. Para poder ver los tensores con las dimensiones deseadas, el objeto `torch.Tensor` contiene varios atributos importantes:*
- *`size`. Nos dice cuantos elementos tiene el tensor en cada dimensión.*
- *`stride`. Nos dice cuantos elementos en memoria tenemos que movernos para obtener el siguiente elemento de una dimensión. Por ejemplo, en la imagen de abajo podemos ver que para obtener el siguiente elemento en la segunda dimensión (columnas) tenemos que movernos un lugar en memoria, y para obtener el siguiente elemento en la primera dimensión (filas) tenemos que movernos tres lugares en memoria.*
- *`offset`. Nos dice cuál es el primer elemento del tensor en la memoria.*
- *`storage`. Nos dice donde se encuentra guardado el tensor en memoria.*
<figure>
    <img src="attachments/tensor-metadata.png" width="900" height="700" align="center" style="display: block; margin: 0 auto;"/>
    <figcaption style="text-align: center;">Stevens, Eli. (2020). Relationship between tensor offser, size, and stride. In Stevens, Eli, <i>Deep Learning with PyTorch</i> (p. 56).</figcaption>
</figure>

## Tensores aleatorios

*Por qué queremos aprender a generar tensores aleatorios? Porque los pesos de las redes neuronales se inicializan de esta manera. Entonces, al comenzar el entrenamiento de nuestra red neuronal debemos pasarle un tensor que represente los pesos iniciales, el cual tenemos que poder generar de manera aleatoria.*

In [20]:
# Tensor de dos dimensiones de tamaño (3, 4)
random_tensor = torch.rand(size=(2, 3))
print(f'random_tensor =\n {random_tensor}')

random_tensor =
 tensor([[0.6386, 0.5047, 0.0468],
        [0.9551, 0.8437, 0.5572]])


In [22]:
print(f'Dimensions: {random_tensor.ndim}')
print(f'Size: {random_tensor.shape}')

Dimensions: 2
Size: torch.Size([2, 3])


In [23]:
random_tensor = torch.rand(size=(10, 2, 3))
print(f'random_tensor =\n {random_tensor}')

random_tensor =
 tensor([[[0.2895, 0.8592, 0.3154],
         [0.7859, 0.4792, 0.2665]],

        [[0.1918, 0.3231, 0.9610],
         [0.7503, 0.5626, 0.2700]],

        [[0.2092, 0.8292, 0.7639],
         [0.6841, 0.6064, 0.9029]],

        [[0.4516, 0.9106, 0.9012],
         [0.2175, 0.6269, 0.4178]],

        [[0.9009, 0.8265, 0.8337],
         [0.9213, 0.0376, 0.2380]],

        [[0.5426, 0.7731, 0.0437],
         [0.2255, 0.9555, 0.9879]],

        [[0.6404, 0.5305, 0.2738],
         [0.4166, 0.6042, 0.3748]],

        [[0.7394, 0.3000, 0.6479],
         [0.0416, 0.6007, 0.4241]],

        [[0.8982, 0.4961, 0.6603],
         [0.4529, 0.2528, 0.1696]],

        [[0.5100, 0.7112, 0.9630],
         [0.6973, 0.5117, 0.5903]]])


In [24]:
print(f'Dimensions: {random_tensor.ndim}')
print(f'Size: {random_tensor.shape}')

Dimensions: 3
Size: torch.Size([10, 2, 3])


*Las imágenes las vamos a representar como tensores de tres dimensiones, en donde la primera dimensión representa los canales de colores (RGB), mientras que la segunda y tercera dimensión representa la cantidad de píxeles de la imágen. Por ejemplo, si tenemos una imagen a color de 1024x1024 y la convertimos en un tensor, entonces tendríamos un tensor de tres dimensiones con tamaño `torch.Size([3, 1024, 1024])`.*

In [25]:
random_image_tensor = torch.rand(size=(3, 1024, 1024))
random_image_tensor

tensor([[[0.4814, 0.0632, 0.9753,  ..., 0.1883, 0.0756, 0.9011],
         [0.3321, 0.1456, 0.4719,  ..., 0.8391, 0.7805, 0.9025],
         [0.8482, 0.3054, 0.4833,  ..., 0.8161, 0.4669, 0.0689],
         ...,
         [0.6489, 0.1121, 0.1003,  ..., 0.7453, 0.2007, 0.9964],
         [0.2765, 0.6053, 0.9880,  ..., 0.5661, 0.5327, 0.3379],
         [0.1350, 0.9351, 0.9738,  ..., 0.9875, 0.6422, 0.8706]],

        [[0.3541, 0.7974, 0.2714,  ..., 0.3374, 0.7037, 0.5539],
         [0.7805, 0.4649, 0.8135,  ..., 0.0389, 0.6538, 0.3782],
         [0.9135, 0.1864, 0.4188,  ..., 0.3658, 0.5919, 0.9976],
         ...,
         [0.8582, 0.4580, 0.7376,  ..., 0.6061, 0.8997, 0.4062],
         [0.8784, 0.2916, 0.4300,  ..., 0.4109, 0.7739, 0.8302],
         [0.6331, 0.1060, 0.0243,  ..., 0.3143, 0.0407, 0.0370]],

        [[0.2824, 0.8100, 0.5089,  ..., 0.4428, 0.2192, 0.8499],
         [0.1997, 0.4274, 0.6938,  ..., 0.2850, 0.7926, 0.9036],
         [0.9079, 0.3626, 0.6193,  ..., 0.9576, 0.1364, 0.

In [27]:
print(f'Dimensiones: {random_image_tensor.ndim}')
print(f'Size: {random_image_tensor.shape}')

Dimensiones: 3
Size: torch.Size([3, 1024, 1024])


*Siempre que trabajemos con procesos aleatorios (o pseudo-aleatorios), vamos a observar diferencias en los resultados. Incluso teniendo la misma semilla, puede que un mismo proceso aleatorio genere distintos resultados si lo ejecutamos en distintos dispositivos (i.e., CPU o GPU). Para más información acerca de como trabajar con procesos no-determinísticos en PyTorch, leer la siguiente [documentación](https://pytorch.org/docs/stable/notes/randomness.html).*

*Podemos definir una semilla que nos permita reproducir los resultados de un experimento utilizando la función `torch.manual_seed()` (se define la misma semilla para todos los dispositivos). Ahora, ejecutar multiples veces el mismo código nos devuelve el mismo resultado.*

In [28]:
torch.manual_seed(42)
random_tensor_1 = torch.rand(size=(1, 5))
random_tensor_2 = torch.rand(size=(1, 5))

print(f'Random tensor 1: {random_tensor_1}')
print(f'Random tensor 2: {random_tensor_2}')

# We can compare two tensors using the function torch.equal() or the operator ==
print(f'Are the tensors equal? {torch.equal(random_tensor_1, random_tensor_2)}')

Random tensor 1: tensor([[0.8823, 0.9150, 0.3829, 0.9593, 0.3904]])
Random tensor 2: tensor([[0.6009, 0.2566, 0.7936, 0.9408, 0.1332]])
Are the tensors equal? False


*Notar que, a pesar de estar utilizando la misma semilla, los tensores que generamos no son iguales entre sí. Para obtener dos tensores iguales deberíamos a ejecutar `torch.manual_seed()` antes de crear cada uno de los tensores.*

In [29]:
torch.manual_seed(42) # Definimos la semilla para el primer tensor
random_tensor_1 = torch.rand(size=(1, 5))
torch.manual_seed(42) # Definimos la semilla para el segundo tensor
random_tensor_2 = torch.rand(size=(1, 5))

print(f'Random tensor 1: {random_tensor_1}')
print(f'Random tensor 2: {random_tensor_2}')

# We can compare two tensors using the function torch.equal() or the operator ==
print(f'Are the tensors equal? {torch.equal(random_tensor_1, random_tensor_2)}')

Random tensor 1: tensor([[0.8823, 0.9150, 0.3829, 0.9593, 0.3904]])
Random tensor 2: tensor([[0.8823, 0.9150, 0.3829, 0.9593, 0.3904]])
Are the tensors equal? True


## Tensor-like objects

In [30]:
range_tensor = torch.arange(start=0, end=1000, step=36)
range_tensor

tensor([  0,  36,  72, 108, 144, 180, 216, 252, 288, 324, 360, 396, 432, 468,
        504, 540, 576, 612, 648, 684, 720, 756, 792, 828, 864, 900, 936, 972])

In [31]:
print(f'Dimension: {range_tensor.ndim}')
print(f'Shape: {range_tensor.shape}')
print(f'Dtype: {range_tensor.dtype}')

Dimension: 1
Shape: torch.Size([28])
Dtype: torch.int64


*Cuando decimos que vamos a crear o que creamos un objeto que es "tensor-like", nos referimos a crear un nuevo tensor que tenga la misma dimensión, tamaño y tipo de dato que otro tensor ya existente, pero con otros valores.*

*Por ejemplo, podemos creamos un nuevo tensor a partir del tensor `range_tensor` que tenga la misma dimensión, tamaño y tipo de dato, pero que la información que contiene sea distinta (solo tenga ceros).*

In [32]:
zeros = torch.zeros_like(input=range_tensor)
zeros

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])

In [33]:
print(f'Dimension: {zeros.ndim}')
print(f'Shape: {zeros.shape}')
print(f'Dtype: {zeros.dtype}')

Dimension: 1
Shape: torch.Size([28])
Dtype: torch.int64


## Atributos de los tensores

### Tipo de dato

*Por defecto, PyTorch crea los tensores utilizando el tipo de dato `torch.float32`. Hay muchos más tipos de datos [disponibles](https://pytorch.org/docs/stable/tensors.html#data-types), que se diferencian principalmente por la precisión númerica que nos aportan. Por ejemplo, el tipo de dato `torch.float64` tiene mucho más precisión numérica que el tipo de dato `torch.float32` (casi el doble), pero consume mucho más memoria.*

*Podemos especificar el tipo de dato que queremos para nuestro tensor utilizando el argumento `dtype`.*

In [34]:
TENSOR_0 = torch.tensor([3., 2., 1.], dtype=torch.float64)
TENSOR_0

tensor([3., 2., 1.], dtype=torch.float64)

In [35]:
TENSOR_0.dtype

torch.float64

*Podemos modificar el tipo de dato utilizando el método `torch.Tensor.type()` o el método `torch.Tensor.to()`. Este último método es el más recomendado, ya que también nos permite modificar otros atributos del tensor.*

In [37]:
# Change tensor's datatype
TENSOR_1 = TENSOR_0.to(dtype=torch.float16) # or use torch.Tensor.to()
TENSOR_1

tensor([3., 2., 1.], dtype=torch.float16)

*Cuando realizamos operaciones matemáticas entre dos tensores con distintos tipos de datos, PyTorch utiliza una **regla de promoción** para definir el tipo de dato del tensor resultante (tendrá el tipo de dato con mayor precisión). Por ejemplo, si multiplicamos un tensor con tipo de dato `torch.float32` y otro con tipo de dato `torch.float64`, el tensor que obtenemos después de la operación es `torch.float64`.*

*Esto es algo importante a tener en cuenta porque:*
1. *Podemos llegar a consumir más memoria de la esperada si las operaciones que realizamos están constantemente incrementando la precisión númerica de nuestros tensores.*
2. *Las operaciones entre tensores con distintos tipos de datos suelen ser menos performantes porque PyTorch también se tiene que encargar de convertir los datos.*
3. *Algunos dispositivos tienen soporte limitado para algunos tipos de datos (e.g., las GPU no soportan tensores con tipos de datos `torch.float64` o las `mps` no soportan los tensores con tipos de datos `torch.float16`).*

In [38]:
TENSOR_0 * TENSOR_1

tensor([9., 4., 1.], dtype=torch.float64)

### Dispositivo

*Podemos también definir el **dispositivo** en donde se guarda y ejecuta el tensor. Hay varios tipos de dispositivos que podemos utilizar, pero los más utilizados son:*
1. *`cpu`: nos permite guardar y ejecutar los tensores en la memoria RAM de nuestra computadora o máquina virtual.*
2. *`cuda`: nos permite guardar y ejecutar los tensores en la GPU. Si tenemos disponibles más de una GPU, podemos especificar cual queremos utilizar, e.g.,  `cuda:0` (primer GPU), `cuda:1` (segunda GPU), `cuda:2` (tercera GPU), etc.*
3. *`mps`: nos permite utilizar la GPU integrada en las computadoras con Apple Silicon (similar a como `cuda` nos permite utilizar las GPUs de NVIDIA). Para más información sobre el uso de MPS con PyTorch, leer la siguiente [documentación](https://pytorch.org/docs/stable/notes/mps.html).*

*Por qué es importante definir el dispositivo?:*
1. *Si tenemos dos tensores que están guardados en dos dispositivos distintos no podemos realizar operaciones entre ellos.*
2. *Mover tensores de un dispositivo a otro puede ser computacionalmente costoso.*
3. *El tipo de dato que tengamos disponible para utilizar depende del dispositivo que utilicemos.*

In [39]:
TENSOR.device

device(type='cpu')

*Podemos elegir el dispositivo en donde queremos guardar y ejecutar el tensor al momento de crearlo:*

In [41]:
# Define a torch.device
torch_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create a new tensor with specified device
TENSOR_2 = torch.rand(size=(3, 1024, 1024), device=torch_device)
print(f'Tensor: {TENSOR_2}')
print(f'Device: {TENSOR_2.device}')

Tensor: tensor([[[0.2137, 0.1287, 0.7578,  ..., 0.5280, 0.1456, 0.7621],
         [0.8253, 0.4947, 0.8110,  ..., 0.7116, 0.7734, 0.5908],
         [0.3095, 0.3278, 0.5857,  ..., 0.7197, 0.9459, 0.6801],
         ...,
         [0.8766, 0.0069, 0.1601,  ..., 0.7396, 0.1457, 0.8224],
         [0.9497, 0.4182, 0.1513,  ..., 0.2655, 0.8717, 0.3968],
         [0.3618, 0.1220, 0.6681,  ..., 0.9665, 0.2104, 0.3793]],

        [[0.5913, 0.7940, 0.0250,  ..., 0.3615, 0.9619, 0.2593],
         [0.3742, 0.9553, 0.4215,  ..., 0.9176, 0.5058, 0.0018],
         [0.3236, 0.1479, 0.7749,  ..., 0.4080, 0.7298, 0.5154],
         ...,
         [0.1911, 0.9423, 0.8503,  ..., 0.5304, 0.1596, 0.3789],
         [0.2172, 0.7798, 0.9750,  ..., 0.1987, 0.9614, 0.8083],
         [0.8856, 0.7259, 0.3923,  ..., 0.7049, 0.5056, 0.8305]],

        [[0.7707, 0.6171, 0.6775,  ..., 0.1210, 0.4782, 0.9287],
         [0.0903, 0.1733, 0.3913,  ..., 0.9720, 0.0357, 0.6195],
         [0.6506, 0.2457, 0.7716,  ..., 0.9107, 0.

*Podemos también utilizar la función `torch.Tensor.to` para cambiar el tipo de dato o el dispositivo de un tensor (notar que es necesario re-asignar el tensor a la variable):*

In [42]:
TENSOR_2 = TENSOR_2.to(device='mps')
print(f'Device: {TENSOR_2.device}')

Device: mps:0


### Gradientes

*Otro argumento importante de los tensores es el argumento `requires_grad`. Cuando definimos `requires_grad=True` en un tensor, le estamos pidiendo a PyTorch que guarde todas las operaciones realizadas sobre ese tensor, para así poder calcular gradientes. Esto es muy útil cuando queremos entrenar un modelo de Deep Learning, ya que nos permite calcular los gradientes de la función de pérdida con respecto a los pesos del modelo utilizando backpropagation.*

*Notar que en el siguiente ejemplo creamos una nueva variable, `y`, que es igual a la suma de los elementos en `X`. Luego, para calcular los gradientes de `y` con respecto a los elementos en `X` (i.e., como cambia el valor de `y` ante un cambio infinitesimal en alguno de los elementos de `X`), llamamos a la función `.backward()` desde la variable `y`. Sin embargo, los gradientes los podemos encontrar en el tensor `X` (eso tiene sentido porque los gradientes nos dicen como cambia `y` cuando cambia un elemento de `X`.*

In [43]:
# Creates a tensor and operates on it
X = torch.tensor([3., 2., 1.], requires_grad=True, device='mps')

# Sum all elements of the tensor and compute the gradients
y = X.sum()
y.backward()
print(f'Gradients after automatic differentiation: {X.grad}')

Gradients after automatic differentiation: tensor([1., 1., 1.], device='mps:0')


## PyTorch y NumPy

*Podemos crear tensores a partir de arrays de NumPy, y viceversa. Esto es muy útil porque podemos utilizar las funcionalidades NumPy para manipular nuestros datos, y luego utilizar los tensores de PyTorch para entrenar nuestros modelos de Deep Learning.*

*Podemos crear un tensor a partir de un array de NumPy utilizando la función `torch.from_numpy(ndarray)`. Si queremos convertir un tensor a un array de NumPy, tenemos que utilizar el método `torch.Tensor.numpy()`.*

In [48]:
array = np.arange(1., 10.).reshape((3, 3))
print(f'Array de NumPy: \n{array}')

Array de NumPy: 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


In [49]:
tensor = torch.from_numpy(array)
print(f'Tensor de PyTorch: \n{tensor}')

Tensor de PyTorch: 
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]], dtype=torch.float64)


*Algo muy importante a tener en cuenta es que el tipo de dato predeterminado por NumPy es diferente al de PyTorch (NumPy utiliza `float64` mientras que PyTorch utiliza `float32`). El tensor que creamos a partir del array de NumPy hereda el tipo de dato del array, lo cual podría generar problemas al querer operar entre tensores (si los tipos de datos no coinciden).*

In [50]:
print(f"Array's datatype: {array.dtype}")
print(f"Tensor's datatype: {tensor.dtype}")

Array's datatype: float64
Tensor's datatype: torch.float64


In [51]:
tensor = tensor.to(dtype=torch.float32)
print(f"Tensor's datatype: {tensor.dtype}")

Tensor's datatype: torch.float32
